<a href="https://colab.research.google.com/github/javierarteagapuell/deeplearning/blob/main/Prac2Ej1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LIBRERIAS mlxtend o apyori

In [283]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets

#pip install mlxtend  

In [284]:
data = pd.read_csv('BlackFriday.csv', encoding='UTF-8')
data.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


In [285]:
data.columns

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3'],
      dtype='object')

In [286]:
data.isnull().values.any() 

True

Vemos en que columnas hay NA's

In [287]:
data.isna().sum()  

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2             72344
Product_Category_3            162562
dtype: int64

Como más de la mitad de las filas de Product Category 3 son NA's eliminamos esta columna

In [288]:
data = data.drop(['Product_Category_3'], axis=1)

Vemos si hay algún dato duplicado

In [289]:
if any(data.duplicated()):
  print("Hay datos duplicados")
else:
  print("No hay datos duplicados")

No hay datos duplicados


Agrupamos los productos por usuario

In [290]:
Tabla_Usuario_Producto = data.groupby('User_ID')['Product_ID'].apply(','.join)
Tabla_Usuario_Producto

User_ID
1000001    P00259342,P00216342,P00157542,P00212942,P00070...
1000002    P00182742,P00102542,P00201442,P00303442,P00364...
1000003    P00025442,P00253142,P00319042,P00289942,P00148...
1000004                                            P00128942
1000005    P00081142,P00132642,P00223542,P00223642,P00108...
                                 ...                        
1006036    P00031042,P00162242,P00062942,P00112742,P00118...
1006037    P00058042,P00209842,P00115842,P00083842,P00059...
1006038    P00120542,P00316642,P00112142,P00131742,P00022...
1006039    P00291842,P00085842,P00019442,P00088242,P00188...
1006040    P00313342,P00179542,P00150042,P00129642,P00100...
Name: Product_ID, Length: 5891, dtype: object

Pasamos los productos a forma boleana

Si el usuario ha comprado ese producto True
sino False

In [291]:
Server_ID = data.groupby(['User_ID','Product_ID'])['Occupation'].sum().apply(lambda x: x>0).unstack().reset_index().fillna(False).set_index('User_ID')
Server_ID.head()

Product_ID,P00000142,P00000242,P00000342,P00000442,P00000542,P00000642,P00000742,P00000842,P00000942,P00001042,...,P0099042,P0099142,P0099242,P0099342,P0099442,P0099542,P0099642,P0099742,P0099842,P0099942
User_ID,,,,,,,,,,,,,,,,,,,,,
1000001,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1000002,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1000003,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1000004,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1000005,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Itemset frecuentes para K==1 (Son los que tengan un soporte alto)

In [292]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(Server_ID, min_support=0.06, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.071295,(P00000142)
1,0.085724,(P00010742)
2,0.105755,(P00025442)
3,0.078085,(P00028842)
4,0.078425,(P00031042)


In [293]:
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


2) Para k≥2.

a. Mostrar los itemset frecuentes candidatos y su soporte.

In [294]:
Server_ID2 = data.groupby(['User_ID','Product_ID'])['Occupation'].sum().apply(lambda x: x>=2).unstack().reset_index().fillna(False).set_index('User_ID')
Server_ID2.head()

Product_ID,P00000142,P00000242,P00000342,P00000442,P00000542,P00000642,P00000742,P00000842,P00000942,P00001042,...,P0099042,P0099142,P0099242,P0099342,P0099442,P0099542,P0099642,P0099742,P0099842,P0099942
User_ID,,,,,,,,,,,,,,,,,,,,,
1000001,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1000002,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1000003,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1000004,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1000005,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [295]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(Server_ID2, min_support=0.06, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.065354,(P00000142)
1,0.078764,(P00010742)
2,0.095230,(P00025442)
3,0.070446,(P00028842)
4,0.068919,(P00031042)


3) Repetir el proceso 2 hasta que no se generen nuevos itemsets frecuentes. CON 8 se para

In [296]:
#Server_ID3 = data.groupby(['User_ID','Product_ID'])['Occupation'].sum().apply(lambda x: x>7).unstack().reset_index().fillna(False).set_index('User_ID')
#frequent_itemsets = apriori(Server_ID3, min_support=0.06, use_colnames=True)
#frequent_itemsets.head()

4) Mostrar todas las posibles reglas con la confianza de cada una de ellas.

In [297]:
#rules = association_rules(frq_items, metric ="lift", min_threshold = 0.8)
#rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

reglas = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8)
reglas.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


5) Listar todas las reglas que sean de alta confianza.

In [298]:
#rules = association_rules(frq_items, metric ="lift", min_threshold = 0.8)
#rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

reglas = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8)
reglas.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


A partir de lo anterior se pide:

- Prueba al menos tres configuraciones de soporte y frecuencia para cada género, edad y tipo de producto.
(1 punto)

- ¿Qué diferencias hay entre usar soporte y frecuencia? Respalda la respuesta con datos

-   Para   una   de   las   configuraciones,   interpreta   algunas   de   las   reglas   que   te   hayan   resultado   interesantes
obtenidas   usando   la   clase  association_rules  y   un   par   de   configuraciones   cambiando   la   variable  metric  y
min_threshold. Justifica los resultados. (1 punto)

In [299]:
reglas = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.8)
reglas.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


- Dadas las mejores configuraciones. ¿Existen reglas o patrones que se repitan? ¿Podemos generalizar  de
alguna manera como se comportan los clientes? Expón las conclusiones respaldadas con datos (1 punto)

In [301]:
import inspect

inspect(sorted(x = reglas, decreasing = True, by = "confidence"))

TypeError: ignored

In [ ]:
reglas_maximales <- reglas [is.maximal(reglas)]
reglas_maximales

In [ ]:
reglas_redundantes <- reglas[is.redundant(x = reglas, measure = "confidence")]
reglas_redundantes

In [ ]:
plot (reglas)